# Nettoyage et préparation du dataset de vins

In [146]:
import pandas as pd
from pandas import isnull

file_path = "../data/raw/winemag-data_first150k.csv"

# Chargement et nettoyage par chunks

In [147]:
chunksize = 20000
clean_chunks = []

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    # Renommage des colonnes
    chunk = chunk.rename(columns={
        "Unnamed: 0": "id",
        "region_1": "primary_region",
        "region_2": "secondary_region",
    })

    # Suppression des lignes complètement vides
    chunk = chunk.dropna(how="all")

    # Suppression des doublons
    chunk = chunk.drop_duplicates()

    clean_chunks.append(chunk)

# Concaténation de tous les chunks
df_clean = pd.concat(clean_chunks, ignore_index=True)

# Suppression finale des doublons sur l'ensemble du dataset
df_clean = df_clean.drop_duplicates()

print(f"Nombre de lignes après nettoyage : {len(df_clean)}")
print(f"\nAperçu des données :")
print(df_clean.head())

Nombre de lignes après nettoyage : 150930

Aperçu des données :
   id country                                        description  \
0   0      US  This tremendous 100% varietal wine hails from ...   
1   1   Spain  Ripe aromas of fig, blackberry and cassis are ...   
2   2      US  Mac Watson honors the memory of a wine once ma...   
3   3      US  This spent 20 months in 30% new French oak, an...   
4   4  France  This is the top wine from La Bégude, named aft...   

                            designation  points  price        province  \
0                     Martha's Vineyard      96  235.0      California   
1  Carodorum Selección Especial Reserva      96  110.0  Northern Spain   
2         Special Selected Late Harvest      96   90.0      California   
3                               Reserve      96   65.0          Oregon   
4                            La Brûlade      95   66.0        Provence   

      primary_region   secondary_region             variety  \
0        Napa Valle

## Supression de la colone secondary region et des lignes avec des valeurs nuls

In [148]:
if "secondary_region" in df_clean.columns:
    df_clean = df_clean.drop("secondary_region", axis=1)
else:
    print('La colone secondary_region n\'existe pas.')

df_clean = df_clean.drop(df_clean[df_clean["designation"].isnull()].index)
df_clean = df_clean.drop(df_clean[df_clean["primary_region"].isnull()].index)
df_clean = df_clean.drop(df_clean[df_clean["price"].isnull()].index)
df_clean = df_clean.drop(df_clean[df_clean["country"].isnull()].index)
df_clean = df_clean.drop(df_clean[df_clean["province"].isnull()].index)

In [149]:
missing_values = df_clean.isnull().sum()
if missing_values.sum() > 0:
    print("\nValeurs manquantes par colonne :")
    missing_percent = (missing_values / len(df_clean)) * 100
    missing_df = pd.DataFrame({
        'Valeurs manquantes': missing_values,
        'Pourcentage': missing_percent
    })
    print(missing_df[missing_df['Valeurs manquantes'] > 0].sort_values('Pourcentage', ascending=False))
else:
    print("Aucune valeurs manquantes.")



Aucune valeurs manquantes.


# Sauvegarde des données nettoyées

In [150]:
df_clean.to_csv("../data/cleaned/wine_data_cleaned.csv", index=False)
print(f"\nDonnées sauvegardées : {len(df_clean)} lignes")



Données sauvegardées : 77284 lignes
